# `DATA MODELING`

## **TOPIC: FILMS ANALYSIS**

`Group ID`: 17

`Group Member`:
- 22127404_Tạ Minh Thư
- 22127359_Chu Thúy Quỳnh
- 22127302_Nguyễn Đăng Nhân

### Import

In [254]:
import pandas as pd
from collections import Counter

### Read data from file

In [255]:
data = pd.read_csv('cleaned_data.csv', sep=",")
data

,Rank,Title,Foreign %,Domestic %,Year,Genre,Director,Writer,Cast
0,1,Avatar,73.1,26.9,2009,"['Action', 'Adventure', 'Fantasy', 'Sci-Fi']",['James Cameron'],['James Cameron'],"['Sam Worthington', 'Zoe Saldana', 'Sigourney ..."
1,2,Avengers: Endgame,69.3,30.7,2019,"['Action', 'Adventure', 'Drama', 'Sci-Fi']","['Anthony Russo', 'Joe Russo']","['Christopher Markus', 'Stephen McFeely', 'Sta...","['Robert Downey Jr.', 'Chris Evans', 'Mark Ruf..."
2,3,Avatar: The Way of Water,70.5,29.5,2022,"['Action', 'Adventure', 'Fantasy', 'Sci-Fi']",['James Cameron'],"['James Cameron', 'Rick Jaffa', 'Amanda Silver...","['Sam Worthington', 'Zoe Saldana', 'Sigourney ..."
3,4,Titanic,70.2,29.8,1997,"['Drama', 'Romance']",['James Cameron'],['James Cameron'],"['Leonardo DiCaprio', 'Kate Winslet', 'Billy Z..."
4,5,Star Wars: Episode VII - The Force Awakens,54.8,45.2,2015,"['Action', 'Adventure', 'Sci-Fi']",['J.J. Abrams'],"['Lawrence Kasdan', 'J.J. Abrams', 'Michael Ar...","['Daisy Ridley', 'John Boyega', 'Oscar Isaac',..."
...,...,...,...,...,...,...,...,...,...
995,996,The Final Destination,64.3,35.7,2009,"['Horror', 'Thriller']",['David R. Ellis'],"['Eric Bress', 'Jeffrey Reddick']","['Nick Zano', 'Krista Allen', 'Andrew Fiscella..."
996,997,Atlantis: The Lost Empire,54.8,45.2,2001,"['Action', 'Adventure', 'Animation', 'Family',...","['Gary Trousdale', 'Kirk Wise']","['Tab Murphy', 'Kirk Wise', 'Gary Trousdale', ...","['Michael J. Fox', 'Jim Varney', 'Corey Burton..."
997,998,Inside Man,52.4,47.6,2006,"['Crime', 'Drama', 'Mystery', 'Thriller']",['Spike Lee'],['Russell Gewirtz'],"['Denzel Washington', 'Clive Owen', 'Jodie Fos..."
998,999,The Waterboy,13.2,86.8,1998,"['Comedy', 'Sport']",['Frank Coraci'],"['Tim Herlihy', 'Adam Sandler']","['Adam Sandler', 'Kathy Bates', 'Henry Winkler..."


### Feature engineering

#### Genre

In the analysis phase, we identified the top 5 genres with the highest revenue.
- First, define a list containing these 5 genres: ["Adventure", "Action", "Comedy","Drama", "Fantasy"].
- For each movie, count how many of its genres match with the top 5 genres.
- Create the new column, which will hold the count of matching genres for each movie.

In [256]:
data['Genre'] = data['Genre'].apply(eval)


top_5_genres = ["Adventure", "Action", "Comedy", "Drama", "Fantasy"]

engineered_data = data.copy()
engineered_data['matching_genres_count'] = engineered_data['Genre'].apply(
    lambda genres: sum(1 for genre in genres if genre in top_5_genres)
)

engineered_data = engineered_data[['Rank', 'Title', 'Year', 'matching_genres_count']]

# Hiển thị DataFrame mới
engineered_data


,Rank,Title,Year,matching_genres_count
0,1,Avatar,2009,3
1,2,Avengers: Endgame,2019,3
2,3,Avatar: The Way of Water,2022,3
3,4,Titanic,1997,1
4,5,Star Wars: Episode VII - The Force Awakens,2015,2
...,...,...,...,...
995,996,The Final Destination,2009,0
996,997,Atlantis: The Lost Empire,2001,3
997,998,Inside Man,2006,1
998,999,The Waterboy,1998,1


#### Main cast popularity

1. The first step is to analyze the list of actors from all movies in the dataset. For each actor that appears, the system counts how many times they have been featured in the dataset. This helps identify the popularity of each actor within the dataset.

2. The next step, for each movie, the system calculates the total number of appearances of the actors in the movie's main cast (the list of actors). This total reflects the level of fame of the main cast, as movies with more famous actors tend to have a higher appeal.

In [257]:

# Đếm số lần diễn viên xuất hiện trong tập dữ liệu
cast_counter = Counter(actor for cast_list in data['Cast'].apply(eval) for actor in cast_list)

# Phổ biến của diễn viên chính
engineered_data['main_cast_popularity'] = data['Cast'].apply(lambda x: sum(cast_counter[actor] for actor in eval(x)))
engineered_data

,Rank,Title,Year,matching_genres_count,main_cast_popularity
0,1,Avatar,2009,3,28
1,2,Avengers: Endgame,2019,3,43
2,3,Avatar: The Way of Water,2022,3,18
3,4,Titanic,1997,1,20
4,5,Star Wars: Episode VII - The Force Awakens,2015,2,14
...,...,...,...,...,...
995,996,The Final Destination,2009,0,4
996,997,Atlantis: The Lost Empire,2001,3,10
997,998,Inside Man,2006,1,21
998,999,The Waterboy,1998,1,21


#### Director

In [258]:
# Chuyển chuỗi thành danh sách đạo diễn
data['Director'] = data['Director'].apply(eval)

# Đếm số lần xuất hiện của mỗi đạo diễn trong toàn bộ bộ phim
director_counter = Counter(director for director_list in data['Director'] for director in director_list)

# Tính tổng số phim mà các đạo diễn tham gia cho mỗi bộ phim
engineered_data['director_movie_count'] = data['Director'].apply(
    lambda directors: sum(director_counter[director] for director in directors)
)

engineered_data


,Rank,Title,Year,matching_genres_count,main_cast_popularity,director_movie_count
0,1,Avatar,2009,3,28,5
1,2,Avengers: Endgame,2019,3,43,8
2,3,Avatar: The Way of Water,2022,3,18,5
3,4,Titanic,1997,1,20,5
4,5,Star Wars: Episode VII - The Force Awakens,2015,2,14,6
...,...,...,...,...,...,...
995,996,The Final Destination,2009,0,4,1
996,997,Atlantis: The Lost Empire,2001,3,10,6
997,998,Inside Man,2006,1,21,1
998,999,The Waterboy,1998,1,21,2


#### Writer

In [259]:
data['Writer'] = data['Writer'].apply(eval)
writer_counter = Counter(writer for writer_list in data['Writer'] for writer in writer_list)
engineered_data['writer_movie_count'] = data['Writer'].apply(lambda writers: sum(writer_counter[writer] for writer in writers))

engineered_data


,Rank,Title,Year,matching_genres_count,main_cast_popularity,director_movie_count,writer_movie_count
0,1,Avatar,2009,3,28,5,11
1,2,Avengers: Endgame,2019,3,43,8,295
2,3,Avatar: The Way of Water,2022,3,18,5,75
3,4,Titanic,1997,1,20,5,11
4,5,Star Wars: Episode VII - The Force Awakens,2015,2,14,6,38
...,...,...,...,...,...,...,...
995,996,The Final Destination,2009,0,4,1,2
996,997,Atlantis: The Lost Empire,2001,3,10,6,31
997,998,Inside Man,2006,1,21,1,1
998,999,The Waterboy,1998,1,21,2,13


#### Director and cast collaborations

In [260]:
director_cast_collaborations = Counter()

for _, row in data.iterrows():
    directors = row['Director']
    cast = row['Cast']

    for director in directors:
        for actor in cast:
            director_cast_collaborations[(director, actor)] += 1

engineered_data['director_cast_collaborations'] = data.apply(
    lambda row: sum(director_cast_collaborations[(director, actor)] 
                    for director in row['Director']
                    for actor in row['Cast']), axis=1
)
engineered_data

,Rank,Title,Year,matching_genres_count,main_cast_popularity,director_movie_count,writer_movie_count,director_cast_collaborations
0,1,Avatar,2009,3,28,5,11,1492
1,2,Avengers: Endgame,2019,3,43,8,295,2196
2,3,Avatar: The Way of Water,2022,3,18,5,75,1436
3,4,Titanic,1997,1,20,5,11,1305
4,5,Star Wars: Episode VII - The Force Awakens,2015,2,14,6,38,1450
...,...,...,...,...,...,...,...,...
995,996,The Final Destination,2009,0,4,1,2,225
996,997,Atlantis: The Lost Empire,2001,3,10,6,31,1518
997,998,Inside Man,2006,1,21,1,1,310
998,999,The Waterboy,1998,1,21,2,13,527


#### Having top 5 writer in each genre

In [261]:
# Bước 1: Tìm biên kịch xuất hiện nhiều nhất trong từng thể loại
from collections import Counter

# Tạo một từ điển để lưu trữ biên kịch phổ biến trong từng thể loại
genre_writer_popularity = {}

# Lặp qua mỗi bộ phim và tính toán sự xuất hiện của biên kịch theo thể loại
for _, row in data.iterrows():
    genres = row['Genre'] # Danh sách thể loại của bộ phim
    writers = row['Writer'] # Danh sách biên kịch của bộ phim
    
    for genre in genres:
        if genre not in genre_writer_popularity:
            genre_writer_popularity[genre] = Counter()
        
        for writer in writers:
            genre_writer_popularity[genre][writer] += 1

top_5_writers_per_genre = {genre: writers.most_common(5) 
                            for genre, writers in genre_writer_popularity.items()}

# Bước 3: Thêm cột kiểm tra sự xuất hiện của biên kịch trong thể loại
engineered_data['has_top_5_writer'] = data.apply(
    lambda row: 1 if any(writer in dict(top_5_writers_per_genre[genre]) for genre in row['Genre'] for writer in row['Writer']) else 0,
    axis=1
)
engineered_data


,Rank,Title,Year,matching_genres_count,main_cast_popularity,director_movie_count,writer_movie_count,director_cast_collaborations,has_top_5_writer
0,1,Avatar,2009,3,28,5,11,1492,0
1,2,Avengers: Endgame,2019,3,43,8,295,2196,1
2,3,Avatar: The Way of Water,2022,3,18,5,75,1436,0
3,4,Titanic,1997,1,20,5,11,1305,0
4,5,Star Wars: Episode VII - The Force Awakens,2015,2,14,6,38,1450,1
...,...,...,...,...,...,...,...,...,...
995,996,The Final Destination,2009,0,4,1,2,225,0
996,997,Atlantis: The Lost Empire,2001,3,10,6,31,1518,0
997,998,Inside Man,2006,1,21,1,1,310,0
998,999,The Waterboy,1998,1,21,2,13,527,0


In [262]:


# Bước 1: Tính toán sự xuất hiện của đạo diễn theo thể loại
genre_director_popularity = {}

# Lặp qua từng bộ phim trong dataset
for _, row in data.iterrows():
    genres = row['Genre'] # Danh sách thể loại của bộ phim
    directors = row['Director']  # Danh sách đạo diễn của bộ phim
    
    # Cập nhật đạo diễn cho từng thể loại
    for genre in genres:
        if genre not in genre_director_popularity:
            genre_director_popularity[genre] = Counter()
        
        for director in directors:
            genre_director_popularity[genre][director] += 1

# Bước 2: Tìm ra top 5 đạo diễn trong mỗi thể loại
top_5_directors_per_genre = {
    genre: {director: count for director, count in directors.most_common(5)} 
    for genre, directors in genre_director_popularity.items()
}

engineered_data['has_top_5_director'] = data.apply(
    lambda row: 1 if any(director in top_5_directors_per_genre[genre]for genre in row['Genre'] for director in row['Director']) else 0,
    axis=1
)


#### Ratio revenue

In [263]:
engineered_data['foreign_to_domestic_ratio'] = data['Foreign %'] / data['Domestic %']
engineered_data

,Rank,Title,Year,matching_genres_count,main_cast_popularity,director_movie_count,writer_movie_count,director_cast_collaborations,has_top_5_writer,has_top_5_director,foreign_to_domestic_ratio
0,1,Avatar,2009,3,28,5,11,1492,0,0,2.717472
1,2,Avengers: Endgame,2019,3,43,8,295,2196,1,0,2.257329
2,3,Avatar: The Way of Water,2022,3,18,5,75,1436,0,0,2.389831
3,4,Titanic,1997,1,20,5,11,1305,0,0,2.355705
4,5,Star Wars: Episode VII - The Force Awakens,2015,2,14,6,38,1450,1,1,1.212389
...,...,...,...,...,...,...,...,...,...,...,...
995,996,The Final Destination,2009,0,4,1,2,225,0,0,1.801120
996,997,Atlantis: The Lost Empire,2001,3,10,6,31,1518,0,0,1.212389
997,998,Inside Man,2006,1,21,1,1,310,0,0,1.100840
998,999,The Waterboy,1998,1,21,2,13,527,0,0,0.152074


## Data preparation